In [2]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns


# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
# Cargar extensión de Kedro
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [3]:
movies_metadata = catalog.load("movies_metadata")
credits = catalog.load("credits")
ratings = catalog.load("ratings")

print(movies_metadata.head())
print(credits.head())
print(ratings.head())

[10/31/25 17:30:08] INFO     Loading data from movies_metadata (CSVDataset)...                 ]8;id=667557;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=212915;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

[10/31/25 17:30:09] INFO     Loading data from credits (CSVDataset)...                         ]8;id=369723;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=201271;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

[10/31/25 17:30:12] INFO     Loading data from ratings (CSVDataset)...                         ]8;id=141506;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=81539;file://C:\Users\ariel\OneDrive\Desktop\Machine\proyect-machine\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497         

In [4]:
# Seleccionar solo columnas numéricas
numeric_cols = movies_metadata.select_dtypes(include=['float64', 'int64'])

# Calcular correlación con vote_count
correlations = numeric_cols.corr()['vote_count'].sort_values(ascending=False)
print(correlations)

vote_count      1.000000
revenue         0.812022
vote_average    0.123607
runtime         0.113539
Name: vote_count, dtype: float64


In [5]:
# 📦 Importar librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ================================================
# 🎬 Modelo KNN Regressor - Movies Metadata
# ================================================

# Convertir columnas numéricas
numeric_cols = ['revenue', 'budget', 'popularity', 'runtime', 'vote_average', 'vote_count']
for c in numeric_cols:
    movies_metadata[c] = pd.to_numeric(movies_metadata[c], errors='coerce')

# Eliminar filas con nulos
movies_metadata = movies_metadata.dropna(subset=numeric_cols + ['status'])

# Codificación de la variable categórica 'status'
status_dummies = pd.get_dummies(movies_metadata['status'], drop_first=True)

# Variables predictoras
X = movies_metadata[['revenue', 'budget', 'popularity', 'runtime', 'vote_average']]
X = pd.concat([X, status_dummies], axis=1)

# Variable objetivo
y = movies_metadata['vote_count']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Escalado de variables (recomendado para KNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔧 Entrenamiento del modelo KNN Regressor
model_knn = KNeighborsRegressor(n_neighbors=10, weights='distance', n_jobs=-1)

model_knn.fit(X_train_scaled, y_train)
y_pred = model_knn.predict(X_test_scaled)

# 📊 Evaluación
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print("RESULTADOS MODELO KNN REGRESSOR")
print("="*50)
print(f"MSE:  {mse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")


RESULTADOS MODELO KNN REGRESSOR
MSE:  65126.6213
MAE:  57.1002
RMSE: 255.1992
R²:   0.7580


In [6]:
# 📦 Importar librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ================================================
# 🎬 Modelo KNN Regressor - Movies Metadata
# ================================================

# Convertir columnas numéricas
numeric_cols = ['revenue', 'budget', 'popularity', 'runtime', 'vote_average', 'vote_count']
for c in numeric_cols:
    movies_metadata[c] = pd.to_numeric(movies_metadata[c], errors='coerce')

# Eliminar filas con nulos
movies_metadata = movies_metadata.dropna(subset=numeric_cols + ['status'])

# Codificación de la variable categórica 'status'
status_dummies = pd.get_dummies(movies_metadata['status'], drop_first=True)

# Variables predictoras
X = movies_metadata[['revenue', 'budget', 'popularity', 'runtime', 'vote_average']]
X = pd.concat([X, status_dummies], axis=1)

# Variable objetivo
y = movies_metadata['vote_count']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Escalado (muy importante para KNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ================================================
# 🔍 GridSearchCV para encontrar mejores parámetros
# ================================================

# Definir el modelo base
knn = KNeighborsRegressor()

# Definir los hiperparámetros a probar
param_grid = {
    'n_neighbors': [3, 5, 10, 15, 20],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 = Manhattan, 2 = Euclidiana
}

# Configurar el GridSearch con validación cruzada k=5
grid_search = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

# Entrenar el modelo
grid_search.fit(X_train_scaled, y_train)

# Mostrar mejores parámetros
print("🔎 Mejores parámetros encontrados:")
print(grid_search.best_params_)

# Mejor modelo encontrado
best_knn = grid_search.best_estimator_

# Predicciones
y_pred = best_knn.predict(X_test_scaled)

# ================================================
# 📊 Evaluación del modelo
# ================================================
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nRESULTADOS MODELO KNN (MEJORADO)")
print("="*50)
print(f"MSE:  {mse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")


Fitting 5 folds for each of 20 candidates, totalling 100 fits
🔎 Mejores parámetros encontrados:
{'n_neighbors': 10, 'p': 1, 'weights': 'distance'}

RESULTADOS MODELO KNN (MEJORADO)
MSE:  61374.9149
MAE:  55.9400
RMSE: 247.7396
R²:   0.7719
